In [1]:
import pycrfsuite
import codecs
import numpy as np
import jieba.posseg as pseg

from sklearn.preprocessing import LabelBinarizer
from itertools import chain
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn

In [2]:
txt_name = ['trigger_train', 'argument_train', 'trigger_test','argument_test']
txt = []
for i in txt_name:
    f = codecs.open(i+'.txt', 'r', 'utf8')
    txt.append(f.readlines())
    f.close()

In [3]:
#txt[0]

In [4]:
'''
def get_corpus(trigger_train, argument_train):
    trigger_train = [each.split() for each in trigger_train]
    argument_train = [each.split() for each in argument_train]
    new_list = []
    in_list = []
    for each1, each2 in zip(trigger_train, argument_train):
        print(each1, each2)
        if each1 != []:
            in_list.append(each1.append(each2[1])) 
        else:
            new_list.append(in_list)
            in_list = []  
    return new_list
train = get_corpus(txt[0], txt[1])
'''

'\ndef get_corpus(trigger_train, argument_train):\n    trigger_train = [each.split() for each in trigger_train]\n    argument_train = [each.split() for each in argument_train]\n    new_list = []\n    in_list = []\n    for each1, each2 in zip(trigger_train, argument_train):\n        print(each1, each2)\n        if each1 != []:\n            in_list.append(each1.append(each2[1])) \n        else:\n            new_list.append(in_list)\n            in_list = []  \n    return new_list\ntrain = get_corpus(txt[0], txt[1])\n'

In [5]:
def get_corpus(data):
    data = [each.split() for each in data]
    new_list = []
    in_list = []
    for each in data:
        if each != []:
            in_list.append(each) 
        else:
            new_list.append(in_list)
            in_list = []  
    return new_list
txt = [get_corpus(each) for each in txt]

In [6]:
txt[0][0]

[['跨', 'O'],
 ['党派', 'O'],
 ['大陆', 'O'],
 ['台商', 'O'],
 ['权益', 'O'],
 ['促进会', 'O'],
 ['6', 'O'],
 ['号', 'O'],
 ['成立', 'T_Business']]

In [30]:
def pos(word):
    p = pseg.cut(word)
    flag = 0
    for i in p:
        pos = i.flag
        flag += 1
    if flag != 1:
        pos = ''
    return pos

def word2features(sent, i):
    word = sent[i][0]
    features = [
        #'bias',
        #'word.length=%s' % len(word),
        'word.isdigit=%s' % word.isdigit(),
        #'word[-1]=' + word[-1],
        #'word[0]=' + word[0],
        'word=' + word,
    ]
    if pos(word) != '':
        features.append('pos=' + pos(word))
        if len(pos(word)) > 1:
            features.extend(['pos[0]=' + pos(word)[0], 'pos[1:]=' + pos(word)[1:]])
#############################
    if i > 0:
        word = sent[i-1][0]
        features.extend([
            #'-1:word.length=%s' % len(word),
            #'-1:word.isdigit=%s' % word.isdigit(),
            '-1:word=' + word,
            #'-1:word[-1]=' + word[-1],
            #'-1:word[0]=' + word[0],
        ])
        if pos(word) != '':
            features.append('-1:pos=' + pos(word))
            if len(pos(word)) > 1:
                features.extend(['-1:pos[0]=' + pos(word)[0], '-1:pos[1:]=' + pos(word)[1:]])
    else:
        features.append('BOS')
##################################
    if i < len(sent)-1:
        word = sent[i+1][0]
        features.extend([
            #'+1:word.length=%s' % len(word),
            #'+1:word.isdigit=%s' % word.isdigit(),
            '+1:word=' + word,
            #'+1:word[-1]=' + word[-1],
            #'+1:word[0]=' + word[0],
        ])
        if pos(word) != '':
            features.append('+1:pos=' + pos(word))
            if len(pos(word)) > 1:
                features.extend(['+1:pos[0]=' + pos(word)[0], '+1:pos[1:]=' + pos(word)[1:]])
    else:
        features.append('EOS')
    return features
'''
#################################
    if i > 1:
        word = sent[i-2][0]
        features.extend([
            #'-2:word.length=%s' % len(word),
            '-2:word.isdigit=%s' % word.isdigit(),
            #'-2:word=' + word,
            #'-2:word[-1]=' + word[-1],
            #'-2:word[0]=' + word[0],
        ])
        if pos(word) != '':
            features.extend(['-2:pos=' + pos(word),'-2:pos[0]=' + pos(word)[0]])
    else:
        features.append('BOS1')
#################################
    if i < len(sent)-2:
        word = sent[i+2][0]
        features.extend([
            #'+2:word.length=%s' % len(word),
            '+2:word.isdigit=%s' % word.isdigit(),
            #'+2:word=' + word,
            #'+2:word[-1]=' + word[-1],
            #'+2:word[0]=' + word[0],
        ])
        if pos(word) != '':
            features.extend(['+2:pos=' + pos(word),'+2:pos[0]=' + pos(word)[0]])
    else:
        features.append('EOS1')
#################################
'''



def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [31]:
%%time
# txt_name = ['trigger_train', 'argument_train', 'trigger_test','argument_test']

x_trigger_train = [sent2features(s) for s in txt[0]]
y_trigger_train = [sent2labels(s) for s in txt[0]]

x_trigger_test = [sent2features(s) for s in txt[2]]
y_trigger_test = [sent2labels(s) for s in txt[2]]

x_argument_train = [sent2features(s) for s in txt[1]]
y_argument_train = [sent2labels(s) for s in txt[1]]

x_argument_test = [sent2features(s) for s in txt[3]]
y_argument_test = [sent2labels(s) for s in txt[3]]

Wall time: 1min


In [32]:
x_trigger_train[3][0]

['word.isdigit=False',
 'word=南斯拉夫',
 'pos=nrt',
 'pos[0]=n',
 'pos[1:]=rt',
 'BOS',
 '+1:word=反对党',
 '+1:pos=n']

In [33]:
def CRF(x_train, y_train, x_test, model_name):
    trainer = pycrfsuite.Trainer(verbose=False)

    for xseq, yseq in zip(x_train, y_train):
        trainer.append(xseq, yseq)

    trainer.set_params({
        'c1': 1.0,   # coefficient for L1 penalty
        'c2': 1e-3,  # coefficient for L2 penalty
        'max_iterations': 50,  # stop earlier

        # include transitions that are possible, but not observed
        'feature.possible_transitions': True
    })

    trainer.train(model_name + '.crfsuite')

    #print(trainer.logparser.last_iteration)
    
    tagger = pycrfsuite.Tagger()
    tagger.open(model_name + '.crfsuite')
    '''
    if model_name == 'trigger':
        i = 2
    else: i = 3 
    example_sent = txt[i][13]
    print(' '.join(sent2tokens(example_sent)), end='\n\n')

    print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
    print("Correct:  ", ' '.join(sent2labels(example_sent)))
    '''
    y_pred = [tagger.tag(xseq) for xseq in x_test]
    return y_pred

In [34]:
%%time
y_trigger_pred = CRF(x_trigger_train, y_trigger_train, x_trigger_test, 'trigger')
y_argument_pred = CRF(x_argument_train, y_argument_train, x_argument_test, 'argument')

Wall time: 15.2 s


In [35]:
def test_example(K, model_name='trigger', txt = txt):
    tagger = pycrfsuite.Tagger()
    tagger.open(model_name + '.crfsuite')

    if model_name == 'trigger':
        i = 2
    else: i = 3 
    example_sent = txt[i][K]
    print(' '.join(sent2tokens(example_sent)), end='\n\n')

    print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
    print("Correct:  ", ' '.join(sent2labels(example_sent)))

In [38]:
K = np.random.randint(len(y_trigger_pred))
test_example(K, model_name='trigger')
test_example(K, model_name='argument')

J 君 的 本 本领 了 好几年 ， 可 就是 没车 开 。 那天 打电话 告诉 我 ， 买

Predicted: O O O O O O O O O O O O O O T_Contact O O O T_Transaction
Correct:   O O O O O O O O O O O O O O O O O O T_Transaction
出售 国营 公司

Predicted: A_Org A_Org A_Org
Correct:   O A_Artifact A_Artifact


In [39]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [40]:
print(bio_classification_report(y_trigger_test, y_trigger_pred))
print(bio_classification_report(y_argument_test, y_argument_pred))

               precision    recall  f1-score   support

   T_Business       0.87      0.65      0.74        31
   T_Conflict       0.71      0.62      0.66       135
    T_Contact       0.97      0.48      0.64        58
    T_Justice       0.73      0.58      0.65       141
       T_Life       0.83      0.75      0.79        72
   T_Movement       0.81      0.68      0.74       146
  T_Personnel       0.91      0.34      0.50        88
T_Transaction       0.54      0.50      0.52        14

  avg / total       0.80      0.59      0.67       685

                     precision    recall  f1-score   support

      A_Adjudicator       0.66      0.54      0.60       144
            A_Agent       0.16      0.23      0.19       190
         A_Artifact       0.14      0.25      0.18       374
         A_Attacker       0.38      0.11      0.17       265
      A_Beneficiary       0.00      0.00      0.00        39
            A_Buyer       0.00      0.00      0.00        12
            A_Crime

D:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
